In [1]:
from models.normal_nets.proxyless_nets import ProxylessNASNets
import json
import torch
import librosa
import numpy as np
import onnx_tf

2024-11-18 07:39:33.695428: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-18 07:39:34.467374: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-18 07:39:34.783977: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-18 07:39:34.855649: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-18 07:39:35.293341: I tensorflow/core/platform/cpu_feature_guar

ModuleNotFoundError: No module named 'keras.src.engine'

In [8]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [2]:
config_file = '/shareddrive/working/keyword_spotting/nas_model_adele/learned_net/net.config'
checkpoints = '/shareddrive/working/keyword_spotting/nas_model_adele/checkpoint/checkpoint.pth.tar'

In [28]:
pad_or_trunc = lambda a,i : a[0:i] if len(a) > i else a if len(a) == i else np.pad(a,(0, (i-len(a))))

In [9]:
with open(config_file,'r') as f:
    net_config = json.load(f)
checkpoint = torch.load(checkpoints,map_location=device )

/tmp/ipykernel_2892/1901895386.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoints,map_location=device )


In [4]:
model = ProxylessNASNets.build_from_config(net_config)

In [14]:
if 'state_dict' in checkpoint:
            checkpoint = checkpoint['state_dict']
    
model.load_state_dict(checkpoint,strict=False)

_IncompatibleKeys(missing_keys=['blocks.0.mobile_inverted_conv.inverted_bottleneck.conv.weight', 'blocks.0.mobile_inverted_conv.inverted_bottleneck.conv.quantized_weight', 'blocks.0.mobile_inverted_conv.inverted_bottleneck.bn.weight', 'blocks.0.mobile_inverted_conv.inverted_bottleneck.bn.bias', 'blocks.0.mobile_inverted_conv.inverted_bottleneck.bn.running_mean', 'blocks.0.mobile_inverted_conv.inverted_bottleneck.bn.running_var', 'blocks.0.mobile_inverted_conv.depth_conv.conv.weight', 'blocks.0.mobile_inverted_conv.depth_conv.conv.quantized_weight', 'blocks.0.mobile_inverted_conv.depth_conv.bn.weight', 'blocks.0.mobile_inverted_conv.depth_conv.bn.bias', 'blocks.0.mobile_inverted_conv.depth_conv.bn.running_mean', 'blocks.0.mobile_inverted_conv.depth_conv.bn.running_var', 'blocks.0.mobile_inverted_conv.point_linear.conv.weight', 'blocks.0.mobile_inverted_conv.point_linear.conv.quantized_weight', 'blocks.0.mobile_inverted_conv.point_linear.bn.weight', 'blocks.0.mobile_inverted_conv.point_l

In [24]:
model.eval()

ProxylessNASNets(
  (melspec): ConvertibleSpectrogram(
    (spec_transf): None
    (mel): MelFilterbank(
      (mel_transform): MelScale()
    )
    (stft): _STFT_Internal()
  )
  (first_conv): ConvLayer(
    (conv): QConv2d(1, 72, kernel_size=(5, 11), stride=(1, 2), padding=(2, 5), bias=False, num_bits=8)
    (bn): BatchNorm2d(72, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU6(inplace=True)
  )
  (blocks): ModuleList(
    (0): MobileInvertedResidualBlock(
      (mobile_inverted_conv): MBInvertedConvLayer(
        (inverted_bottleneck): Sequential(
          (conv): QConv2d(72, 216, kernel_size=(1, 1), stride=(1, 1), bias=False, num_bits=8)
          (bn): BatchNorm2d(216, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act): ReLU6(inplace=True)
        )
        (depth_conv): Sequential(
          (conv): QConv2d(216, 216, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), groups=216, bias=False, num_bits=8)
          (bn): 

In [26]:
with torch.inference_mode():
    o = model(torch.tensor([[-0.1,-0.5],[-0.1,-0.5]]))
o

tensor([[-0.3554,  0.3258,  0.0401],
        [-0.3554,  0.3258,  0.0401]])

In [34]:
audio_file = '/shareddrive/working/data_code/data/adele/trimmed/Adele_1.wav'
waveform,_ = librosa.load(audio_file,sr=44100)
waveform = pad_or_trunc(waveform,44100)
waveform.shape

(44100,)

In [39]:
with torch.inference_mode():
    o = model((torch.tensor(waveform).unsqueeze(0).unsqueeze(0)))
o

tensor([[-0.3523,  0.3235,  0.0397]])

In [47]:
 torch.onnx.export(model, torch.tensor(waveform).unsqueeze(0).unsqueeze(0), 'adele.onnx', opset_version=11)